In [1]:
## using pre-processing from https://www.kaggle.com/code/yasirakyzl/covid-19-ml-model-90-accuracy

In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
DATA_FOLDER = 'data/covid/'

In [3]:
df = pd.read_csv(DATA_FOLDER + 'covid_data.csv')

In [4]:
df = df[(df.PNEUMONIA == 1) | (df.PNEUMONIA == 2)]
df = df[(df.DIABETES == 1) | (df.DIABETES == 2)]
df = df[(df.COPD == 1) | (df.COPD == 2)]
df = df[(df.ASTHMA == 1) | (df.ASTHMA == 2)]
df = df[(df.INMSUPR == 1) | (df.INMSUPR == 2)]
df = df[(df.HIPERTENSION == 1) | (df.HIPERTENSION == 2)]
df = df[(df.OTHER_DISEASE == 1) | (df.OTHER_DISEASE == 2)]
df = df[(df.CARDIOVASCULAR == 1) | (df.CARDIOVASCULAR == 2)]
df = df[(df.OBESITY == 1) | (df.OBESITY == 2)]
df = df[(df.RENAL_CHRONIC == 1) | (df.RENAL_CHRONIC == 2)]
df = df[(df.TOBACCO == 1) | (df.TOBACCO == 2)]
df['DEATH'] = [1 if each=='9999-99-99' else 0 for each in df.DATE_DIED]
df.PREGNANT = df.PREGNANT.replace(97,2)
df = df[(df.PREGNANT == 1) | (df.PREGNANT == 2)]
df.drop(columns=['INTUBED','ICU','DATE_DIED'], inplace=True)
for i in df.columns:
    print(i,"=>\t",len(df[i].unique()))

USMER =>	 2
MEDICAL_UNIT =>	 13
SEX =>	 2
PATIENT_TYPE =>	 2
PNEUMONIA =>	 2
AGE =>	 121
PREGNANT =>	 2
DIABETES =>	 2
COPD =>	 2
ASTHMA =>	 2
INMSUPR =>	 2
HIPERTENSION =>	 2
OTHER_DISEASE =>	 2
CARDIOVASCULAR =>	 2
OBESITY =>	 2
RENAL_CHRONIC =>	 2
TOBACCO =>	 2
CLASIFFICATION_FINAL =>	 7
DEATH =>	 2


## Split by domains (MEDICAL_UNIT)

In [6]:
domain_col = 'MEDICAL_UNIT'
label_col = 'DEATH'

In [7]:
np.unique(np.array(df[domain_col]).astype(str), return_counts=True)

(array(['1', '10', '11', '12', '13', '2', '3', '4', '5', '6', '7', '8',
        '9'], dtype='<U21'),
 array([   151,   7538,   5553, 589209,    811,    159,  18646, 306967,
          7049,  37791,    870,  10091,  37142]))

In [8]:
clean_domains = {
    3: '1',
    4: '2',
    6: '3',
    8: '4',
    12: '5',
}

In [9]:
x_train, x_test, y_train, y_test = {}, {}, {}, {}
## Shuffle whole dataframe before splitting between train and test data afterwards
df = df.sample(frac=1).reset_index(drop=True)
for domain, clean_domain in clean_domains.items():
    domain_train_data = df.loc[df[domain_col] == domain].drop(domain_col, axis=1).head(800)
    domain_test_data = df.loc[df[domain_col] == domain].drop(domain_col, axis=1).tail(2000)
    domain_train_labels = domain_train_data[label_col]
    domain_test_labels = domain_test_data[label_col]
    domain_train_data = domain_train_data.drop(label_col, axis=1)
    domain_test_data = domain_test_data.drop(label_col, axis=1)
    x_train[clean_domain] = np.array(domain_train_data).astype(float)
    x_test[clean_domain] = np.array(domain_test_data).astype(float)
    ## Put min of each column at 0
    for col in range(x_train[clean_domain].shape[-1]):
        x_train[clean_domain][:, col] -= np.min(x_train[clean_domain][:, col])
        x_test[clean_domain][:, col] -= np.min(x_test[clean_domain][:, col])
    y_train[clean_domain], y_test[clean_domain] = np.array(domain_train_labels), np.array(domain_test_labels)

In [10]:
with open(DATA_FOLDER + 'preprocessed_covid.pickle', 'wb') as handle:
    pickle.dump((x_train, x_test, y_train, y_test), handle, protocol=pickle.HIGHEST_PROTOCOL)